Obtain Breitbart URL's

In [14]:
# Import necessary libraries
import time
import requests
import urllib
import lxml.html
from bs4 import BeautifulSoup
import newspaper
import trafilatura
from datetime import datetime
import re
from wayback import WaybackClient, memento_url_data
import itertools
import datetime

In [30]:
def obtain_page_urls(
    page="1", base_url="https://www.breitbart.com/politics/", archive=False
):
    """
    Obtain the urls for a page of results from the politics section of Breitbart
    Inputs:
    -page(str): page of results to search in results
    -base_url
    -archive(bool): True if the webpage searched is from the Internet Archive
    Return:
    links_list(list): List of urls
    """
    url = base_url + "page/" + page + "/"
    print(url)
    resp = requests.get(url)
    root = lxml.html.fromstring(resp.text)
    articles_elements = root.cssselect("article")
    links_list = []
    for article in articles_elements:
        url = article.cssselect("h2")[0].cssselect("a")[0].get("href")
        if archive:
            full_link = url[url.find("https://www.breitbart") :]
        else:
            full_link = "breitbart.com" + str(url)

        links_list.append(full_link)

    year = re.search(r"\d{4}", links_list[-1]).group()

    return links_list, int(year)

In [32]:
def obtain_politics_url(year=2022):
    """ """
    links_dict = {}
    links_list = []
    page = 1
    article_year = year
    print("article year", article_year)
    base_url = "https://www.breitbart.com/politics/"
    archive = False

    if year < 2023:
        date = datetime.datetime(year + 1, 1, 1)
        client = WaybackClient()
        results = client.search(
            "https://www.breitbart.com/politics", match_type="exact", from_date=date
        )
        record = next(results)
        base_url = record.view_url
        archive = True

    while article_year == year:
        print("Obtaining results for page", page)
        page_links, article_year = obtain_page_urls(str(page), base_url, archive)
        print("Article year", article_year)
        key = str(year) + ", page " + str(page)
        links_dict[key] = page_links
        links_list.extend(page_links)
        page += 1
        time.sleep(0.5)

    print(len(links_list))

    return links_dict


obtain_politics_url(2022)

article year 2022
2023-01-01 00:00:00
Obtaining results for page 1
https://web.archive.org/web/20230101075905/https://www.breitbart.com/politics/page/1/
Article year 2023
40


{'2022, page 1': ['https://www.breitbart.com/the-media/2023/07/12/nolte-transphobic-cnn-misgenders-transvestite-mr-dylan-mulvaney/',
  'https://www.breitbart.com/tech/2023/07/12/blue-origin-blunder-jeff-bezos-rocket-company-suffers-engine-explosion-during-testing/',
  'https://www.breitbart.com/tech/2023/07/12/never-upset-the-alphabet-people-st-philips-college-fires-prof-for-making-lgbt-student-uncomfortable/',
  'https://www.breitbart.com/politics/2023/07/12/joe-manchin-attending-no-labels-town-hall-new-hampshire-2024-speculation-rises/',
  'https://www.breitbart.com/tech/2023/07/12/survey-majority-of-americans-say-chinas-tiktok-is-a-threat-to-national-security/',
  'https://www.breitbart.com/border/2023/07/12/camo-wearing-migrants-captured-in-arizona-desert-near-border/',
  'https://www.breitbart.com/europe/2023/07/12/zelensky-hails-significant-security-victory-as-nato-summit-ends/',
  'https://www.breitbart.com/politics/2023/07/12/speaker-kevin-mccarthy-raises-62-million-to-expand-h